In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

%config IPCompleter.greedy=True

## UPLOAD DATASETS

In [129]:
path = '../../'

In [130]:
name_cat = 'cat_eda_results.pickle'
name_num = 'num_eda_results.pickle'

In [131]:
cat = pd.read_pickle(path + '/02_Data/03_Work/' + name_cat)
num = pd.read_pickle(path + '/02_Data/03_Work/' + name_num)

# target  separation

In [132]:
target = num[['compra']].copy().reset_index(drop=True)

## CATEGORICAL TRANSFORMATION

### One Hot Encoding

#### Columns to  apply  OHE

In [133]:
var_ohe = ['origen','fuente','ult_actividad','ambito','ocupacion','descarga_lm']

#### Training and apply

In [134]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

In [135]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Save as dataframe

In [136]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

## NUMERICAL TRANSFORMATION

In [137]:
df = pd.concat([cat_ohe,num.reset_index()],axis = 1) #unify datasets with the same rows number.

## RESCALE VARIABLES

###  Min-Max

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4814 entries, 0 to 4813
Data columns (total 44 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4814 non-null   float64
 1   origen_Landing Page Submission            4814 non-null   float64
 2   origen_Lead Add Form                      4814 non-null   float64
 3   origen_OTHER                              4814 non-null   float64
 4   fuente_Chat                               4814 non-null   float64
 5   fuente_Direct Traffic                     4814 non-null   float64
 6   fuente_Google                             4814 non-null   float64
 7   fuente_OTHER                              4814 non-null   float64
 8   fuente_Organic Search                     4814 non-null   float64
 9   fuente_Reference                          4814 non-null   float64
 10  ult_actividad_Chat Conversation     

#### Variables a reescalar con Min-Max

In [139]:
var_mms = df.iloc[:,39:].columns

In [140]:
var_mms

Index(['visitas_total', 'tiempo_en_site_total', 'paginas_vistas_visita',
       'score_actividad', 'score_perfil'],
      dtype='object')

#### Fit and apply

In [141]:
mms = MinMaxScaler()
df_mms = mms.fit_transform(df[var_mms])

#### Save as  dataframe

In [142]:
#Add siffix to the variables
name_mms = [variable + '_mms' for variable in var_mms]

#Save as dataframe
df_mms = pd.DataFrame(df_mms,columns = name_mms)

## JOIN THE REESCALE DATASETS

In [143]:
id = df.id

In [144]:
df_tablon = pd.concat([id,cat_ohe,df_mms,target],axis = 1).set_index('id')

## SAVE DATASETS AFTER DATA TRANFORMATION

In [145]:
path_df_tablon = path + '/02_Data/03_Work/' + 'df_tablon.pickle'
df_tablon.to_pickle(path_df_tablon)